In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier
import keras_tuner as kt


In [54]:
X_train_full = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

y_train_full = X_train_full["label"]
X_train_full = X_train_full.drop("label", axis=1)

X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test/255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_valid, num_classes)



In [93]:
def build_model(hp):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten())
    hp_units_1 = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units_1, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss="categorical_crossentropy",
                metrics=['accuracy'])
    return model



In [94]:
tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=10, overwrite=True, directory="my_dir", project_name="helloworld")
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train, epochs=50, validation_data=(X_valid, y_val))
             
model = tuner.get_best_models()[0]


Trial 30 Complete [00h 00m 13s]
val_accuracy: 0.9696000218391418

Best val_accuracy So Far: 0.9811999797821045
Total elapsed time: 00h 02m 16s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [97]:
predictions = pd.DataFrame(model.predict(X_test))

print(model.summary())

predict_class = np.argmax(predictions, axis=1)

output = pd.DataFrame({"ImageId": X_test.index+1, "Label": predict_class})

output.to_csv("NN_submission.csv", index=False)


875/875 [==============================] - 0s 458us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (32, 784)                 0         
                                                                 
 dense (Dense)               (32, 448)                 351680    
                                                                 
 dense_1 (Dense)             (32, 10)                  4490      
                                                                 
Total params: 356170 (1.36 MB)
Trainable params: 356170 (1.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
